# OpenAI Module Demo

In [ ]:
import mlrun
import pandas as pd

In [ ]:
project = mlrun.get_or_create_project("fastapi-openai",user_project=True,context="./src")

In [ ]:
app = mlrun.import_module("hub://openai")

In [ ]:
app.OpenAIModule.deploy()